In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from skopt.space import Real, Integer, Categorical
from skopt import gp_minimize

import utils.dev_config as dev_conf
import utils.preprocessing as prep

In [2]:
dirs = dev_conf.get_dev_directories("../dev_paths.txt")
unified_dsets = ["unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data"]
matrisome_list = f"{dirs.data_dir}/matrisome/matrisome_hs_masterlist.tsv"

In [3]:
i = 0

In [4]:
matrisome_df = prep.load_matrisome_df(matrisome_list)

In [5]:
rand = np.random.RandomState()

# Load and filter survival data

In [6]:
event_code = {"Alive": 0, "Dead": 1}
covariate_cols = ["figo_stage", "age_at_diagnosis", "race", "ethnicity"]
dep_cols = ["vital_status", "survival_time"]
cat_cols = ["race", "ethnicity", "figo_chr"]
survival_df = prep.load_survival_df(f"{dirs.data_dir}/{unified_dsets[i]}/survival_data.tsv", event_code)

In [7]:
filtered_survival_df = (
    prep.decode_figo_stage(survival_df[["sample_name"] + dep_cols + covariate_cols].dropna(), to="c")
        .query("vital_status == 1")
        .drop(["vital_status"], axis=1)
        .pipe(pd.get_dummies, columns=cat_cols)
        .reset_index(drop = True)
)
filtered_survival_df.columns = filtered_survival_df.columns.str.replace(' ', '_')

print(filtered_survival_df.shape)
# filtered_survival_df.head()

(66, 16)


# Load normalized matrisome count data

In [8]:
norm_matrisome_counts_df = pd.read_csv(f"{dirs.data_dir}/{unified_dsets[i]}/norm_matrisome_counts.tsv", sep='\t')
norm_filtered_matrisome_counts_t_df = prep.transpose_df(
    norm_matrisome_counts_df[["geneID"] + list(filtered_survival_df.sample_name)], "geneID", "sample_name"
)
print(norm_filtered_matrisome_counts_t_df.shape)
# norm_filtered_matrisome_counts_t_df.head()

(66, 1009)


# Join survival and count data

In [9]:
joined_df = (
    pd.merge(filtered_survival_df, norm_filtered_matrisome_counts_t_df, on="sample_name")
        .set_index("sample_name")
)
print(joined_df.shape)
# joined_df.head()

(66, 1023)


In [10]:
rand.seed(123)
shuffled_df = joined_df.sample(frac=1, random_state=rand)
x_df = shuffled_df.iloc[:, 1:]
y_df = shuffled_df.iloc[:, [0]]

In [11]:
def objective(h_params, X, y):
    print(h_params)
    model = GradientBoostingRegressor(
        loss=h_params[0],
        learning_rate=h_params[1],
        n_estimators=h_params[2],
        max_depth=h_params[3],
        max_features=h_params[4],
        alpha=h_params[5]
    )
    # Standardize all variables (except one-hots)
    # Don't really need to do this for decision tree methods, but might as well
    # repeat the procedure that we're doing for the SVMs, just for posterity
    pipeline = make_pipeline(
        ColumnTransformer([
            ("standard", StandardScaler(), ["age_at_diagnosis"] + list(norm_filtered_matrisome_counts_t_df.columns[1:]))
        ], remainder="passthrough"),
        model
    )
    ttr = TransformedTargetRegressor(regressor=pipeline, transformer=StandardScaler())
    return -np.mean(cross_val_score(
        ttr,
        X,
        y,
        cv=KFold(n_splits=5),
        n_jobs=-1,
        scoring="neg_mean_absolute_error"
    ))

In [12]:
space = [
    # We skip "ls" since we're optimizing for mean absolute error in the h_param opt.
    Categorical(["lad", "huber", "quantile"], name="loss"),
    Real(1e-3, 1e-1, name="learning_rate"),
    Integer(int(1e2), int(1e3), name="n_estimators"),
    Integer(2, 5, name="max_depth"),
    Categorical(["auto", "sqrt", "log2"], name="max_features"),
    Real(0.5, 0.9, name="alpha"),
]
n_initial = 20 * (len(space[0].categories) + len(space[4].categories))
n_calls = 100 * (len(space[0].categories) + len(space[4].categories))
# n_initial = 1 * (len(space[0].categories) + len(space[4].categories))
# n_calls = 1 * (len(space[0].categories) + len(space[4].categories))

In [13]:
def save_callback(res):
    # Does this model correspond with the best score seen so far?
    if res.func_vals[-1] == res.fun:
        h_param_df = pd.DataFrame({
            "param": res.space.dimension_names + ["loss_achieved"],
            "param_value": res.x + [res.fun]
        })
        h_param_df.to_csv("opt_gbr_h_params.tsv", sep="\t", index=False)

In [14]:
res = gp_minimize(
    lambda h_ps: objective(h_ps, x_df, y_df),
    space,
    verbose=True,
    random_state=rand,
    n_initial_points=n_initial,
    n_calls = n_calls,
    n_jobs=-1,
    callback=save_callback
)

Iteration No: 1 started. Evaluating function at random point.
['quantile', 0.04860832066690728, 818, 3, 'log2', 0.5039029398683332]
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 1.8118
Function value obtained: 598.3643
Current minimum: 598.3643
Iteration No: 2 started. Evaluating function at random point.
['lad', 0.06923792863971574, 882, 3, 'log2', 0.6073125230846899]
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 1.4791
Function value obtained: 582.0191
Current minimum: 582.0191
Iteration No: 3 started. Evaluating function at random point.
['quantile', 0.027780951796428976, 577, 3, 'log2', 0.8564436383356643]
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 1.0927
Function value obtained: 625.8955
Current minimum: 582.0191
Iteration No: 4 started. Evaluating function at random point.
['huber', 0.04993730401749343, 291, 4, 'auto', 0.8676227203709375]
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 3.8367

/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 130 ended. Search finished for the next optimal point.
Time taken: 2.3732
Function value obtained: 521.8209
Current minimum: 501.8241
Iteration No: 131 started. Searching for the next optimal point.
['huber', 0.001, 100, 2, 'sqrt', 0.5]


/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 131 ended. Search finished for the next optimal point.
Time taken: 2.5935
Function value obtained: 520.8088
Current minimum: 501.8241
Iteration No: 132 started. Searching for the next optimal point.
['huber', 0.001, 100, 2, 'log2', 0.5]


...

Iteration No: 239 ended. Search finished for the next optimal point.
Time taken: 11.1929
Function value obtained: 496.9366
Current minimum: 494.2596
Iteration No: 240 started. Searching for the next optimal point.
['huber', 0.001, 451, 4, 'auto', 0.5]


/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 240 ended. Search finished for the next optimal point.
Time taken: 10.7589
Function value obtained: 496.4293
Current minimum: 494.2596
Iteration No: 241 started. Searching for the next optimal point.
['lad', 0.005205900522753769, 100, 2, 'log2', 0.7042503792985934]
Iteration No: 241 ended. Search finished for the next optimal point.
Time taken: 5.2270
Function value obtained: 521.3356
Current minimum: 494.2596
Iteration No: 242 started. Searching for the next optimal point.
['huber', 0.001, 449, 4, 'auto', 0.5]


/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 242 ended. Search finished for the next optimal point.
Time taken: 10.3858
Function value obtained: 494.8944
Current minimum: 494.2596
Iteration No: 243 started. Searching for the next optimal point.
['lad', 0.005376146483172785, 100, 2, 'log2', 0.7021721875143674]
Iteration No: 243 ended. Search finished for the next optimal point.
Time taken: 5.8662
Function value obtained: 517.9847
Current minimum: 494.2596
Iteration No: 244 started. Searching for the next optimal point.
['lad', 0.007154770364517053, 100, 2, 'log2', 0.6697316478073897]
Iteration No: 244 ended. Search finished for the next optimal point.
Time taken: 5.5682
Function value obtained: 519.7461
Current minimum: 494.2596
Iteration No: 245 started. Searching for the next optimal point.
['lad', 0.005427271632268898, 100, 2, 'log2', 0.7036530719271078]
Iteration No: 245 ended. Search finished for the next optimal point.
Time taken: 5.7810
Function value obtained: 515.0321
Current minimum: 494.2596
Iteration No: 